In [1]:
import numpy as np

In [4]:
def calculate_profit(x1,x2,y,z,n0,p1,p2,p_product):
    #参数说明
    #n:零件1的数量/零件2的数量
    #p1:零件1的次品率
    #p2:零件2的次品率
    #p_product:成品的次品率
    #一些内置的参数
    C1=1#检测零件1的成本
    C2=1#检测零件2的成本
    C_market=56#单件成品的市场售价
    C_assemble=6#装配单件成品的费用
    C_test_product=2#检测单件成品所需的费用
    C_lose=30#调换一件产品引起的损失
    C_disassemble=5#拆解一件产品所需的费用
    #更新次品率
    p1_update=(1-x1)*p1
    p2_update=(1-x2)*p2
    #成品的总次品率
    P=(1-p1_update)*(1-p2_update)*p_product+p1_update+p2_update-p1_update*p2_update
    n=np.floor(min((1-x1*p1)*n0,(1-x2*p2)*n0))
    profit=C_market*(1-P)*n-x1*C1*n0-x2*C2*n0-C_assemble*n-y*C_test_product*n-(1-y)*C_lose*n*P-z*C_disassemble*n*P
    #更新零件数
    if z==0:
        n_update=0
    elif z==1:
        n_update=np.floor(n*P)
    return profit,n_update,p1_update,p2_update

In [5]:
n0=10000
p1=0.2
p2=0.2
p_product=0.2
S=0
same=0
different=0
for p1 in np.linspace(0.95*p1,1.05*p1,5):
    for p2 in np.linspace(0.95*p2,1.05*p2,5):
        for p_product in np.linspace(0.95*p_product,1.05*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[1,1,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,0,1,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为118698476.39102483
第一阶段的生产决策为:
[0, 0, 0, 1, 73611.31000000004, 4685.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 34486.89873500002, 2195.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 1, 0, 61998.72, 0, 0.0, 0.0]
2
最终的最大利润为119448618.8462353
第一阶段的生产决策为:
[0, 0, 0, 1, 70626.05500000002, 4718.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 33321.372749, 2226.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 1, 0, 62403.23999999999, 0, 0.0, 0.0]
3
最终的最大利润为120100624.96120031
第一阶段的生产决策为:
[0, 0, 0, 1, 67640.79999999999, 4751.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 32136.144080000005, 2257.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 1, 0, 62756.40000000001, 0, 0.0, 0.0]
4
最终的最大利润为120725665.5359197
第一阶段的生产决策为:
[0, 0, 0, 1, 64655.545000000035, 4784.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 30931.212728000013, 2288.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 1, 0, 63095.56, 0, 0.0, 0.0]
5
最终的最大利润为121323648.06495987
第一阶段的生产决策为:
[0, 0, 0, 1, 61670.28999999999, 4816.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 29700.411663999985, 2319.0, 0.19, 0.19]
第三阶段的生产决

In [6]:
print(f"在0.95-1.05波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.95-1.05波动时,决策差异率为:{different/125}")

在0.95-1.05波动时,生产决策方案和原方案一致的次数为:0
在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:125
在0.95-1.05波动时,决策差异率为:1.0


In [7]:
n0=10000
p1=0.2
p2=0.2
p_product=0.2
S=0
same=0
different=0
for p1 in np.linspace(0.9*p1,1.1*p1,5):
    for p2 in np.linspace(0.9*p2,1.1*p2,5):
        for p_product in np.linspace(0.9*p_product,1.1*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[1,1,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,0,1,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为112163178.57752013
第一阶段的生产决策为:
[0, 0, 0, 1, 91744.87999999995, 4486.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 41156.75316799998, 2012.0, 0.18000000000000002, 0.18000000000000002]
第三阶段的生产决策为:
[1, 1, 1, 0, 58506.07999999999, 0, 0.0, 0.0]
2
最终的最大利润为113691468.28017972
第一阶段的生产决策为:
[0, 0, 0, 1, 85626.03999999994, 4553.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 38985.53601199997, 2073.0, 0.18000000000000002, 0.18000000000000002]
第三阶段的生产决策为:
[1, 1, 1, 0, 59328.64, 0, 0.0, 0.0]
3
最终的最大利润为115111848.09600025
第一阶段的生产决策为:
[0, 0, 0, 1, 79507.20000000001, 4620.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 36732.32639999999, 2134.0, 0.18000000000000002, 0.18000000000000002]
第三阶段的生产决策为:
[1, 1, 1, 0, 60095.20000000001, 0, 0.0, 0.0]
4
最终的最大利润为116554882.50752027
第一阶段的生产决策为:
[0, 0, 0, 1, 73388.3599999999, 4688.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 34404.46316799995, 2197.0, 0.18000000000

In [8]:
print(f"在0.9-1.1波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.9-1.1波动时,决策差异率为:{different/125}")

在0.9-1.1波动时,生产决策方案和原方案一致的次数为:0
在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:125
在0.9-1.1波动时,决策差异率为:1.0


In [ ]:
n0=10000
p1=0.2
p2=0.2
p_product=0.2
S=0
same=0
different=0
for p1 in np.linspace(0.8*p1,1.2*p1,5):
    for p2 in np.linspace(0.8*p2,1.2*p2,5):
        for p_product in np.linspace(0.8*p_product,1.2*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[1,1,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,0,1,0]):
                same+=1
            else:
                different+=1

In [9]:
print(f"在0.8-1.2波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.8-1.2波动时,决策差异率为:{different/125}")

在0.8-1.2波动时,生产决策方案和原方案一致的次数为:0
在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:125
在0.8-1.2波动时,决策差异率为:1.0
